In [45]:
import boto3
import io
import json
import os
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_colwidth', 100)

### Functions to get shards from s3

In [8]:
def read_parquet_from_s3(filepath, bucket='s-laion', s3=boto3.resource('s3')):
    buffer = io.BytesIO()
    object = s3.Object(bucket, filepath)
    object.download_fileobj(buffer)
    df = pd.read_parquet(buffer)

    return df

def get_filepaths(bucket='s-laion', prefix="vit-h-14-embeddings/metadata", s3=boto3.resource('s3')):
    bucket = s3.Bucket(bucket)
    filepaths = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]

    return filepaths

In [42]:
filepaths = get_filepaths()
index = 10
output_dir = "/fsx/home-siddhesh1793/data/laion_2b_jsonl"
output_filename = filepaths[index].split('/')[-1].replace("parquet", "jsonl")
dataset_name = "laion_2b"

In [56]:
df = read_parquet_from_s3(filepaths[index])

In [65]:
def convert_parquet_to_jsonl(df, output_dir, output_filename, dataset_name):
	df['text_list'] = df['caption'].apply(lambda x: ["This",  f"has the caption {x}"])
	df['image_info'] = df.apply(lambda x: [{"image_name": x['key'], "raw_url": x['url'], "matched_text_index": 1, "NSFW": x['NSFW'], "similarity_matrix": [x['similarity']], "width": x['width'], "height": x['height']}], axis=1)
	df["shard_index"] = index
	df["index"] = df.reset_index().index
	df["dataset"] = dataset_name
	df["modality"] = "image"
	df = df[['text_list', 'image_info', 'shard_index', 'index', 'dataset', 'modality']]
	json_df = json.loads(df.to_json(orient="records"))
	with open(os.path.join(output_dir, output_filename), 'w') as outfile:
		for entry in tqdm(json_df):
			json.dump(entry, outfile)
			outfile.write('\n')

In [66]:
convert_parquet_to_jsonl(df, output_dir, output_filename, dataset_name)

100%|██████████| 10/10 [00:00<00:00, 4778.20it/s]
